# Imports

In [ ]:
from modules.data import  data_reader, data_loader
from modules.sr import result_saver
from modules.models import lag_llama
from modules.experiment.tscv import get_tscv_results, get_summary, extract_metrics
from modules.visualization import graphs
from modules.fine_tuning import lag_llama_ft
from modules.models import prpht


In [4]:
from modules.data import data_loader

a = data_loader.get_data(type = "fx", ticker = "USD/GBP", frequency = "weekly", start = "2019-01-01", end = "2024-01-01", rtrn = True)
#a = data_loader.get_data(type = "stock", ticker = "IBM", frequency = "daily", start = "01-01-2023", end = "01-01-2024")

In [6]:
from modules.models import timegpt
pr = timegpt.get_timegpt_forecast(data=a, prediction_length=1, frequency="weekly", ft_steps=100)

C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\nixtlats\__init__.py:5: FutureWarning: This package is deprecated, please install nixtla instead.
  warnings.warn("This package is deprecated, please install nixtla instead.", category=FutureWarning)
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: W-SUN
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [7]:
pr

[-0.002209331840276718]

In [24]:
import requests
import os
api = os.getenv("ALPHAVANTAGE_API")

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NATURAL_GAS&interval=1min&apikey={api}&month=2019-01-01'
r = requests.get(url)
data = r.json()

print(data)

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.'}


In [20]:
from modules.sr import result_saver

result_saver.save_results(a, "test", type="evaluation")

Data has been written to 'c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\evaluation\test'.


In [ ]:
import os

# this is for renaming files

def rename_files_in_folder(folder_path, old_string, new_string):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            if old_string in filename:
                # Create the new filename by replacing the old string with the new string
                new_filename = filename.replace(old_string, new_string)
                
                # Construct the full file paths
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = os.path.join(folder_path, new_filename)
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                print(f"Renamed: {filename} -> {new_filename}")
            else:
                print(f"No match found in: {filename}")
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "evaluation"))

s1 = "TSCV_R"
s2 = "TSCVR"

rename_files_in_folder(folder_path, s1, s2)
rename_files_in_folder(folder_path_2, s1, s2)

In [39]:
# this is for adding something at the end

import os

# this is for renaming files

def rename_files_in_folder(folder_path):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = old_file_path[:-4] + "__BS=5__ME=4.csv"
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "evaluation"))



rename_files_in_folder(folder_path)
rename_files_in_folder(folder_path_2)

Renaming process completed.
Renaming process completed.


In [1]:
from modules.data import data_reader
import pandas as pd
start_date = "2022-01-01"
end_date = "2024-01-01"

a = data_reader.read_data([""], "data/interest_rate")


In [1]:
from modules.data import data_loader, data_reader
import pandas as pd
start_date = "2019-01-01"
end_date = "2024-01-01"


x_df = data_loader.get_exogenous_data(start_date=start_date, end_date=end_date)

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:123: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values


In [2]:
from modules.data import data_loader

data = data_loader.get_data(type = "index", ticker = "S&P 500", frequency = "monthly", start = "2019-01-01", end = "2024-01-01", rtrn = True)
#a = data_loader.get_data(type = "stock", ticker = "IBM", frequency = "daily", start = "01-01-2023", end = "01-01-2024")

[*********************100%%**********************]  1 of 1 completed


In [3]:
from modules.models import timegpt
pr = timegpt.get_timegpt_forecast(data=data, prediction_length=1, frequency="monthly", ft_steps=10, x=x_df)

C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\nixtlats\__init__.py:5: FutureWarning: This package is deprecated, please install nixtla instead.
  warnings.warn("This package is deprecated, please install nixtla instead.", category=FutureWarning)
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint.

In [4]:
pr

[0.009347334659264498]

In [10]:
from nixtlats import TimeGPT
from nixtla import NixtlaClient

nixtla_client = NixtlaClient(
    #api_key=os.getenv("NIXTLA_API")
    api_key = "nixtla-tok-co7tFFNHZUZfr0Q1FS7VlWmnzoZpWD3x0eVwRx6U3gCUWOoYkpMSf3BJC9fySpa96kb3QdDUPdIpPE9V"
)

nixtla_client.forecast(df = data, h = 1, freq = "M", time_col="ds", target_col="y")["ds"]

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


0    2024-01-01
Name: ds, dtype: object

# Parameters

In [ ]:
# experiment parameters
#constraint: FOLDS x PREDICTION_LENGTH + TRAIN_SIZE !!must not!! be bigger than len(data)

PREDICTION_LENGTH = 1 # currentlz only works for PREDICTION_LENGTH > 1
TICKER = "S&P 500"
FREQUENCY = "minutely" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama", "autoregressor", "fine-tuned Llama"] # currentlz works onlz for these two
FOLDS = 5 # for TSCV # reduced to two for testing purposes
CONTEXT_LENGTH = 245 # set to 245 for testing purposes
METRICS = ['r2', 'mse', 'mae', 'rmse', 'mda', "mape"]

# fine-tuning parameters
BATCH_SIZE = 10
MAX_EPOCHS = 5

# data parameters
START_DATE = "2024-07-22"
END_DATE = "2024-07-23"

# want to add
#TRAIN_PERIOD = # context lenghts. Should take a look into this
TRAIN_SIZE = CONTEXT_LENGTH



# Data

In [ ]:
"""
data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
#if len(data) > 1:
data = data[0]

simple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])
#if len(simple_data) > 1:
simple_data = simple_data[0]

train_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])
train_data = train_data[0]

test_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])
test_data = test_data[0]
"""

In [ ]:
DATA_CONFIG = {"ticker" : TICKER,
               "frequency" : FREQUENCY,
               "start" : START_DATE,
               "end" : END_DATE}



In [ ]:
data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=DATA_CONFIG)

In [ ]:
data

In [ ]:
import 

# Additional parameters

In [ ]:
DATA_LENGTH = len(data)
FOLDS = int((DATA_LENGTH - TRAIN_SIZE) / PREDICTION_LENGTH) # this calculates max ammount of folds we can have given a set TRAIN_SIZE and PREDICTION_LENGTH
FOLDS = 5


In [ ]:
from modules.data import data_loader

In [ ]:
config = {"ticker" : "USD/GBP",
          "frequency" : "daily",
          "start" : "2024-07-01",
          "end" : "2024-07-30"}

In [ ]:
import os
print(os.getenv("ALPHAVANTAGE_API"))

In [ ]:
os.getenv("ALPHAVANTAGE_API")

In [ ]:
os.environ

In [ ]:
for key, value in os.environ.items():
    print(f'{key}: {value}')

In [ ]:
h = data_loader.get_data(data_type="exchange_rate", kwargs=config)

In [ ]:
h

# Fine tuning

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

In [ ]:
r, p = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY,
                           predictor=predictor)

# Saving results

In [ ]:
r

In [ ]:
p

In [ ]:
experiment_name = f"PREDICTION_LENGTH={PREDICTION_LENGTH}__TICKER={TICKER}__FREQUENCY={FREQUENCY}__TYPE_OF_DATA={TYPE_OF_DATA}__FOLDS={FOLDS}__CONTEXT_LENGTH/TRAIN_SIZE={CONTEXT_LENGTH}__FT_START_DATE={FT_START_DATE}__START_DATE={START_DATE}__END_DATE={END_DATE}__FT_LENGTH={FT_LENGTH}__DATA_LENGTH={DATA_LENGTH}" 

In [ ]:
experiment_name = f"P_L={PREDICTION_LENGTH}__T={TICKER}__FR={FREQUENCY}__T_O_D={TYPE_OF_DATA}__FO={FOLDS}__C_L_T_S={CONTEXT_LENGTH}__FT_S_D={FT_START_DATE}__S_D={START_DATE}__E_D={END_DATE}__FT_L={FT_LENGTH}__D_L={DATA_LENGTH}.csv"

In [ ]:
experiment_name

In [ ]:
result_saver.save_results(r, experiment_name, type="evaluation")
result_saver.save_results(p, experiment_name, type="prediction")

# Models

In [ ]:
# arima


In [ ]:
# lag llama

# Fine tuning an estimator

In [ ]:
PREDICTION_LENGTH = 5

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

# SKLearn TSCV

In [ ]:
r, p, a = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY, 
                           predictor=predictor)

In [ ]:
#creating the summary for each model
s = [get_summary(r[i]) for i in range(len(r))]

In [ ]:
means_df, medians_df, stds_df = extract_metrics(s, MODELS)

In [ ]:
means_df

# Visualisation

In [ ]:
graphs.standard_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.interactive_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.prediction_visualisation(MODELS, p, a)

LABEL the graph
label the axes with timestamps
show the history of the actual
table of comparison, not just graph